# Objectives

1. Plot RTT: Regression
1. Plot Rate Variation: Histogram
1. Plot Timetree: Phylo
1. Plot Divtree: Phylo
1. Plot Marginal Distribution: Violin

---
# Setup

## Import Modules

In [1]:
import os
import copy

import seaborn as sns
import matplotlib.pyplot as plt

import random
import json # Read clock model data
import scipy.stats # Regression line
import math

from Bio import Phylo

import dill
from treetime.treetime import plot_vs_years

## Input File Paths

In [2]:
# jupyter notebook output
tree_df_dill = "../../results/clock_model/clock_model.df.obj"
tree_dill = "../../results/clock_model/clock_model.treetime.obj"
augur_json = "../../results/clock_model/clock_model_augur.json"

outdir = "../../results/clock_model/"

# Create output directory if it doesn't exist
if not os.path.exists(outdir):
    os.mkdir(outdir)

## Variables

In [3]:
from config import *

NAME_COL = "Name"
SCRIPT_NAME = "clock_model"

plt.rc('lines', linewidth=2)
plt.rc('legend', labelspacing=0.75)
plt.rc('legend', frameon=False) # legend frame

sns.set_style("ticks")

## Import Dataframe

In [4]:
with open(tree_df_dill, "rb") as infile:
    tree_df = dill.load(infile)
tree_df

,UFboot,sCF,Branch_Length,Branch_Support_Color,coord_x,coord_y,Strain,Date,DateBP,Country,Province,CountryLat,CountryLon,ProvinceLat,ProvinceLon,Biovar,Branch_Major,Branch_Minor,BioSample,BioSampleComment,Branch_Number,timetree_rate,timetree_rate_fold_change,timetree_mutation_length,timetree_raw_date,timetree_num_date,timetree_num_date_confidence,timetree_clock_length,timetree_coord_x,timetree_coord_y,timetree_reg_x,timetree_reg_y,timetree_reg_bad
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NODE0,NA,NA,0,black,0,17.8205,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,9.371838e-09,1.000000,2.148815e-07,-4080-09-12,-4079.302414,"[-4461.973030042604, -3848.8570874035286]",0.100000,0.100000,16.189925,-4079.302414,0.000000,False
SAMEA3541827,NA,NA,3.8504e-06,grey,3.8504e-06,1,RISE509,[-2876:-2677],[-4897:-4698],Russia,Krasnoyarsk Krai,64.6863,97.7453,63.3234,97.098,Bronze Age,0.PRE,0.PRE2,SAMEA3541827,KEEP: SRA Ancient,0,2.693733e-09,0.287428,3.224965e-06,-2889-02-05,-2888.901739,"[-2875.9999999999995, -2676.9999999999995]",1190.400674,1190.500674,1.000000,-2776.500000,0.000003,False
NODE1,100,85,1.32708e-05,black,1.32708e-05,3.15625,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1.317250e-08,1.405541,7.960968e-06,-3487-05-12,-3486.640332,"[-3716.318503809583, -3292.757749728942]",592.662082,592.762082,3.156250,-3486.640332,0.000008,False
SAMEA104233046,NA,NA,2.2484e-06,grey,1.55192e-05,2,Gyvakarai1,[-2621:-2472],[-4642:-4493],Lithuania,Panevezys County,55.35,23.75,55.9156,25.0312,Bronze Age,0.PRE,0.PRE1,SAMEA104233046,KEEP: SRA Ancient,0,2.444031e-09,0.260785,2.150193e-06,-2599-04-20,-2598.700463,"[-2620.9999999999995, -2500.694094889415]",887.939868,1480.701950,2.000000,-2546.500000,0.000010,False
NODE2,100,94.6,1.36364e-05,black,2.69072e-05,4.3125,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1.977152e-08,2.109674,1.443313e-05,-2759-12-09,-2758.062213,"[-2876.7743790999175, -2670.002251893907]",728.578119,1321.340201,4.312500,-2758.062213,0.000022,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NODE_0000366,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,8.935846e-09,0.953479,3.203124e-20,1950-09-17,1950.712034,"[1949.5994918971774, 1952.2028647001257]",1.652304,6030.114447,481.875000,1950.712034,0.000060,False
NODE_0000367,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,8.994253e-09,0.959711,3.203124e-20,1952-04-28,1952.323979,"[1952.0580304793036, 1952.4994493162133]",1.611946,6031.726393,482.750000,1952.323979,0.000060,False
NODE_0000368,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,9.003700e-09,0.960719,3.203124e-20,1953-10-28,1953.823191,"[1952.710862629663, 1954.0]",1.499211,6033.225604,483.500000,1953.823191,0.000060,False


## Import Tree

In [5]:
with open(tree_dill, "rb") as infile:
    tt = dill.load(infile)
tt.tree.ladderize(reverse=False)

In [6]:
tt.clock_model

{'slope': 9.371838111665203e-09,
 'intercept': 4.151561242414281e-05,
 'chisq': 594.3307545143728,
 'hessian': array([[ 7.33532466e+18, -8.52062207e+14],
        [-8.52062207e+14,  7.22904875e+11]]),
 'cov': array([[1.57952199e-19, 1.86172626e-16],
        [1.86172626e-16, 1.60274290e-12]]),
 'r_val': 0.5507397794066982,
 'valid_confidence': True}

---
# 1. Plot RTT: Regression

In [ ]:
regression = tt.clock_model

# Setup axis
fig, (ax1) = plt.subplots(1, dpi=dpi, figsize=figsize_mini)


# Make separate datasets for tips and internal nodes
tip_data = tree_df[(tree_df["timetree_reg_x"] != NO_DATA_CHAR) &
                   (tree_df["Branch_Support_Color"] == TERM_COL) ]

int_data = tree_df[(tree_df["timetree_reg_x"] != NO_DATA_CHAR) & (tree_df["Branch_Support_Color"] != TERM_COL)]

# Time to most recent common ancestor (x-intercept)
t_mrca = -regression['intercept']/regression['slope']

# This does not filter for bad branches!
time_span = np.max(tip_data["timetree_reg_x"]) - np.min(tip_data["timetree_reg_x"])
x_vals = np.array([max(np.min(tip_data["timetree_reg_x"]), t_mrca) - 0.1*time_span, 
                   np.max(tip_data["timetree_reg_x"]+0.05*time_span)])

res = scipy.stats.linregress(list(tip_data["timetree_reg_x"]),list(tip_data["timetree_reg_y"]))

# When no confidence is enabled
dev_rtt = None
dev_slope = None

# Plot internal nodes
ax1.scatter(data=int_data, 
            x="timetree_reg_x", 
            y="timetree_reg_y",
            label="Internal Nodes",
            color=DISC_CMAPLIST[2],
            edgecolor="black",
            linewidth=0.5,
            s=15,
            alpha=0.75,
            zorder=1,)
# Plot tips
ax1.scatter(data=tip_data, 
            x="timetree_reg_x", 
            y="timetree_reg_y",
            label="Tips",
            color=DISC_CMAPLIST[1],
            edgecolor="black",
            linewidth=0.5,
            s=15,
            alpha=0.75,
            zorder=2,)

# Regression line
ax1.plot(x_vals, 
         regression['slope']*x_vals + regression['intercept'],
         label = r"$y=\alpha + \beta t$"+"\n"+
                r"$\beta=$%1.2e"%(regression["slope"])
                + ("+/- %1.e"%dev_slope if dev_slope else "") +
                "\nR²={:.2f}".format(res.rvalue**2) +
                "\nRoot Date: %1.1f"%(-regression['intercept']/regression['slope']) +
                ("+/- %1.2f"%dev_rtt if dev_rtt else ""),
         color=DISC_CMAPLIST[0],
         zorder=3,
        )

# Labels
ax1.set_xlabel("Date")
ax1.set_ylabel("Root-to-Tip Distance")
ax1.set_title("Root-to-Tip Regression on Date")
# Legend
#plt.legend(loc=2, frameon = 1, framealpha=1)
plt.legend(bbox_to_anchor=(0,1.02), 
           loc="upper left",
           frameon = 1, 
           framealpha=1)

# Save
out_path = os.path.join(outdir, SCRIPT_NAME + "_rtt." + FMT) 
plt.savefig(out_path, 
            dpi=dpi, 
            bbox_inches = "tight")

---
# 2. Plot Rate Variation: Histogram

In [ ]:
# Setup axis
fig, ax1 = plt.subplots(1, dpi=dpi, figsize=figsize_mini)

# Plot the histogram
# troubleshoot the disappearing nodes!

min_gamma = ['', 1]
max_gamma = ['', 1]
gamma_dict = {}
    
for rec in tip_data.iterrows():
    sample = rec[0]
    data = rec[1]
    g = tip_data["timetree_rate_fold_change"][sample]
    if g <= min_gamma[1]:
        min_gamma[0] = sample
        min_gamma[1] = g
    if g >= max_gamma[1]:
        max_gamma[0] = sample
        max_gamma[1] = g
    gamma_dict[sample] = g

# Check if there's no rate variation
if min_gamma[1] == 1 and max_gamma[1] == 1:
    hist_bins=1
    ax1.set_xticks([1.0])
else:
    int_max_gamma = math.ceil(max_gamma[1])
    hist_bins = [x/10 for x in range(0,int_max_gamma * 10, 5)]
    xticks_major = [x/10 for x in range(0, int_max_gamma * 10, 5)]
    ax1.set_xticks(xticks_major)
    ax1.set_xlim(0,max_gamma[1])

sns.histplot(x = list(gamma_dict.values()), 
         ax=ax1,
         alpha=0.75,
         bins=hist_bins,
         ) 

ax1.set_xlabel("Rate Multiplier")
ax1.set_ylabel("Number of Branches")
ax1.set_title("Rate Variation Across Branches")

# Save
out_path = os.path.join(outdir, SCRIPT_NAME + "_rate-variation." + FMT) 
plt.savefig(out_path, 
        dpi=dpi, 
        bbox_inches = "tight")

# 3. Plot Timetree: Phylo

In [ ]:
tt_copy = copy.deepcopy(tt)

# Setup plot
plt.rc("lines", linewidth=0.5)
#DIV = 10
#STEP = (CURRENT_YEAR - tt_copy.tree.root.numdate) / DIV
#STEP = round(STEP, -3)

if hasattr(tt.tree.root, "marginal_inverse_cdf"):
    fig, ax1 = plot_vs_years(tt_copy,
                             #step=STEP,  
                             confidence=(1.0-CONFIDENCE,CONFIDENCE), 
                             label_func = lambda x:"",
                             do_show=False)
else:
    fig, ax1 = plot_vs_years(tt_copy,
                             #step=STEP,  
                             label_func = lambda x:"",
                             do_show=False)

ax1.set_title("Time Tree", fontsize=LG_FONT)

# Save
out_path = os.path.join(outdir, SCRIPT_NAME + "_timetree-default." + FMT) 
plt.savefig(out_path, dpi=dpi, bbox_inches = "tight")


## Manual

In [ ]:
# --------------------------------------------
# Setup
plt.rc("lines", linewidth=0.25)
fig, ax1 = plt.subplots(1, dpi=dpi)

# --------------------------------------------
# Data
tt_copy.branch_length_to_years()

# reset color
for c in tt_copy.tree.find_clades():
    c.color = "black"
Phylo.draw(tt_copy.tree,
           axes=ax1, 
           show_confidence=False, 
           label_func = lambda x:'', 
           do_show=False)


# --------------------------------------------
# Ticks
ticks_dict = convert_timetree_ticks(tree=tt_copy.tree, step=500)
# Set the new tick locations and labels
ax1.set_xticks(ticks_dict["tick_locs"])
ax1.set_xticklabels(ticks_dict["tick_labels"])
date_to_x = float(ticks_dict["tick_locs"][-1]) - float(ticks_dict["tick_labels"][-1])


if hasattr(tt_copy.tree.root, "marginal_inverse_cdf"):
    for rec in tree_df.iterrows():
        conf = rec[1]["timetree_num_date_confidence"]
        y = rec[1]["timetree_coord_y"]
        conf_lower = conf[0] + date_to_x
        conf_upper = conf[1] + date_to_x
        ax1.plot([conf_lower, conf_upper], [y,y], alpha=0.3, c="blue", linewidth=3)  


# --------------------------------------------
# Lines
# Vertical lines for dates
for loc in ticks_dict["tick_locs"]:
    ax1.axvline(x=loc, color="darkgrey", alpha=0.5)

# --------------------------------------------
# Labels
ax1.set_xlabel("Year")
ax1.set_ylabel("")
ax1.set_yticks([])
plt.title("Time Tree")

# --------------------------------------------
# Limits
x_buffer = math.ceil((ticks_dict["tick_locs"][-1] - ticks_dict["tick_locs"][0]) * 0.05)
y_buffer = math.ceil(len(tt_copy.tree.get_terminals()) * 0.05)

# No x buffer on lower end
ax1.set_xlim(ticks_dict["tick_locs"][0], ticks_dict["tick_locs"][-1] + x_buffer)
ax1.set_ylim(len(tt_copy.tree.get_terminals()) + y_buffer, 0 - y_buffer)

# --------------------------------------------
# Misc
ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)
ax1.spines["left"].set_visible(False)

# Save
out_path = os.path.join(outdir, SCRIPT_NAME + "_timetree." + FMT) 
plt.savefig(out_path, dpi=dpi, bbox_inches = "tight")

# 4. Plot Divtree: Phylo

In [ ]:
# Set branch length to mutations
for n in tt_copy.tree.find_clades():
    n.branch_length=n.mutation_length
    
min_gamma = min(tree_df["timetree_rate_fold_change"])
max_gamma = max(tree_df["timetree_rate_fold_change"])
gamma_range = max_gamma - min_gamma

GAMMA_COL_PAL = {
    0: "#000000", # black
    1: "#facb35", # yellow
    2: "#ff8800", # orange
    3: "#ff0000"} # red

for c in tt_copy.tree.find_clades():
    if not hasattr(c.branch_length_interpolator, "gamma"):
        c.color = GAMMA_COL_PAL[0]
        continue
    gamma = c.branch_length_interpolator.gamma
    # Switch Case
    if gamma >= 0 and gamma < 1:
        c.color = GAMMA_COL_PAL[0]
    elif gamma >= 1 and gamma < 2:
        c.color = GAMMA_COL_PAL[1]    
    elif gamma >= 2 and gamma < 3:
        c.color = GAMMA_COL_PAL[2]     
    elif gamma >= 3:
        c.color = GAMMA_COL_PAL[3] 

# Setup plot
plt.rc("lines", linewidth=1)
fig, ax1 = plt.subplots(1, dpi=dpi, figsize=figsize, constrained_layout=True,)

Phylo.draw(tt_copy.tree,
           axes=ax1, 
           show_confidence=False, 
           label_func = lambda x:'', 
           do_show=False)

# --------------------------------------------
# Lines
# Vertical lines for dates
for loc in ticks_dict["tick_locs"]:
    ax1.axvline(x=loc, color="darkgrey", alpha=0.5)

# --------------------------------------------
# Labels
ax1.set_xlabel("Branch Length")
ax1.set_ylabel("")
ax1.set_yticks([])
plt.title("Divergence Tree")

# --------------------------------------------
# Limits
coord_x = [x for x in tree_df["coord_x"] if x != NO_DATA_CHAR]
x_buffer = max(coord_x) * 0.05
y_buffer = math.ceil(len(tt_copy.tree.get_terminals()) * 0.01)

ax1.set_xlim(0 - x_buffer, max(coord_x) + x_buffer)
ax1.set_ylim(len(tt_copy.tree.get_terminals()) + y_buffer, 0 - y_buffer)

# --------------------------------------------
# Misc
ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)
ax1.spines["left"].set_visible(False)

# Save
out_path = os.path.join(outdir, SCRIPT_NAME + "_divtree-rate." + FMT) 
plt.savefig(out_path, dpi=dpi, bbox_inches = "tight")

## Demo

In [ ]:
from matplotlib import cm
tt_copy = copy.deepcopy(tt)

fig, ax1 = plt.subplots(1, dpi=dpi)
vmin, vmax = 0.5, 1.5 # color branches according to the rate deviation
for n in tt_copy.tree.find_clades():
    if n.up:
        n.color = [int(x*255) for x in cm.RdBu_r((min(max(vmin, n.branch_length_interpolator.gamma),vmax)-vmin)/(vmax-vmin))[:3]]
    else:
        n.color = [200,200,200]

Phylo.draw(tt_copy.tree, 
           axes=ax1, 
           show_confidence=False, 
           label_func = lambda x:'')

ax1.set_title("Rate Variation")

# ? Add Tree Dataframe as Comments

## Convert num_date_confidence to num_date_bar

Units for num_date_bar are YBP

In [14]:
tree_df["timetree_num_date_bar"] = [NO_DATA_CHAR for row in range(0,len(tree_df))]  
tt_copy = copy.deepcopy(tt)

for rec in tree_df.iterrows():
    node = rec[0]
    conf = tree_df["timetree_num_date_confidence"][node]
    bar_lower = CURRENT_YEAR - conf[0]
    bar_upper = CURRENT_YEAR - conf[1]
    if bar_lower < 10000:
        #print(bar_lower, bar_upper)
        tree_df.at[node, "timetree_num_date_bar"] = [bar_lower, bar_upper]
    else:
        tree_df.at[node, "timetree_num_date_bar"] = [0, 500]
        #tree_df.at[node, "timetree_num_date_bar"] = [bar_lower, bar_upper]
    
metadata_to_comment(tt_copy.tree, tree_df)
Phylo.write(tt_copy.tree, "test_bar.nexus", 'nexus', format_branch_length='%1.{}f'.format(BRANCH_LEN_SIG_DIG))

8043310.013824928 8035460.252670132
7805408.033027759 7797562.567366511
7516527.574196506 7508687.324026325
7386079.696023429 7378241.795265536


1

In [19]:
tree_df.iloc[[0]]

,UFboot,sCF,Branch_Length,Branch_Support_Color,coord_x,coord_y,Strain,Date,DateBP,Country,Province,CountryLat,CountryLon,ProvinceLat,ProvinceLon,Biovar,Branch_Major,Branch_Minor,BioSample,BioSampleComment,Branch_Number,timetree_rate,timetree_rate_fold_change,timetree_mutation_length,timetree_raw_date,timetree_num_date,timetree_num_date_confidence,timetree_clock_length,timetree_coord_x,timetree_coord_y,timetree_reg_x,timetree_reg_y,timetree_reg_bad,timetree_num_date_bar
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NODE0,NA,NA,0,black,0,17.8205,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,9.371838e-09,1.0,2.148815e-07,-4080-09-12,-4079.302414,"[-4461.973030042604, -3848.8570874035286]",0.1,0.1,16.189925,-4079.302414,0.0,False,"[6482.973030042604, 5869.857087403529]"
